In [ ]:
# Visualizing spending habits with data taken from id card
# Started 12/22/2019
# Jack Liu

In [2]:
# Necessary Imports
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# Get HTML file
with open("Transaction History.html") as fp:
    soup = BeautifulSoup(fp)

In [4]:
rows = soup.find('div', id='divHist').find('tbody').find_all('tr')

In [148]:
def process_row(row):
    cell = rows[i].find_all('td')
    if len(cell) == 5:
        post_date.append(cell[0].string.split()[0])
        post_time.append(cell[0].string.split()[1])
        loc.append(cell[1].string)
        typ.append(cell[3].string)
        amt.append(cell[4].string)
    if len(cell) == 3:
        post_date.append(None)
        loc.append(None)
        post_time.append(None)
        typ.append(cell[1].string)
        amt.append(cell[2].string)

In [149]:
def process_location(s):
    t = s.lower()
    if 'wash' in t:
        return 'Washer'
    elif 'dryer' in t:
        return 'Dryer'
    elif 'castle' in t:
        return 'The Castle'
    elif 'O-Hill Aero' in s:
        return 'O-Hill Dining Hall'
    elif 'Runk Dining Aero' in s:
        return 'Runk Dining Hall'
    elif 'FFC' in s:
        return 'Newcomb Dining Hall'
    elif 'crossroads' in t:
        return 'Crossroads'
    elif 'Newcomb Dining To Go' in s:
        return 'N2Go'
    elif 'Einsteins' in s:
        return 'Einsteins Bagels'
    elif 'chick' in t:
        return 'Chick-Fil-A'
    elif 'chik' in t:
        return 'Chick-Fil-A'
    elif 'subway' in t:
        return 'Subway'
    elif 'five guys' in t:
        return 'Five Guys'
    elif 'dumpling' in t:
        return 'Dumpling Food Truck'
    elif 'ValuPort' in s:
        return 'O-Hill Deposit'
    else:
        return s

In [150]:
post_date = []
post_time = []
loc = []
typ = []
amt = []

for i in range(1,len(rows)):
    process_row(rows[i])

In [160]:
data = {'Date':post_date, 'Time':post_time, 'Location':loc, 'Type':typ, 'Amount':amt}
df = pd.DataFrame(data).fillna(method='ffill')
df['Adjusted_Loc'] = df['Location'].apply(process_location)
df.head(10)

,Date,Time,Location,Type,Amount,Adjusted_Loc
0,08/21/2019,16:40:21,Import Loaction,Meal,-50,Import Loaction
1,08/21/2019,16:40:21,Import Loaction,Debit,$-150.00,Import Loaction
2,08/21/2019,16:49:48,Import Loaction,Meal,-10,Import Loaction
3,08/23/2019,12:22:01,OHill ValuPort III,Debit,$-20.00,O-Hill Deposit
4,08/23/2019,12:22:08,OHill ValuPort III,Debit,$-20.00,O-Hill Deposit
5,08/23/2019,12:22:23,OHill ValuPort III,Debit,$-20.00,O-Hill Deposit
6,08/23/2019,12:25:51,O-Hill Aero 1 Loc 473,Meal,1,O-Hill Dining Hall
7,08/23/2019,18:41:53,O-Hill Aero 1 Loc 473,Meal,1,O-Hill Dining Hall
8,08/24/2019,08:32:16,O-Hill Aero 1 Loc 473,Meal,1,O-Hill Dining Hall
9,08/24/2019,14:07:13,ChickFilA (4414),Debit,$7.49,Chick-Fil-A


In [161]:
df['Location'].value_counts()

O-Hill Aero 1 Loc 473             131
FFC Southside Loc 482              48
FFC Northside Loc 483              25
Runk Dining Aero 2 Loc 479         17
Einsteins Bagels - Rice Hall       13
Crossroads Register 1              12
Newcomb Dining To Go Loc 109       11
Castle 4723 (1)                     8
Einsteins Bagels  Rice Hall         8
Chik-Fil-A - Tapingo                6
O-Hill CROSSROADS 1                 6
Einsteins Rice - Tapingo - New      5
Castle 4725 (3)                     4
GibbonDryer10                       3
Import Loaction                     3
OHill ValuPort III                  3
O-Hill Aero 2 Loc 474               3
GibbonWash1                         2
GibbonDryer12                       2
Old - Castle 1 Aero Loc 192         2
GibbonWash5                         2
Dumplings Mobile Vendor             2
GibbonWash3                         2
ChickFilA (4428)                    2
GibbonWash2                         2
ChickFilA (4414)                    2
GibbonDryer9

In [162]:
df['Adjusted_Loc'].value_counts()

O-Hill Dining Hall           134
Newcomb Dining Hall           74
Einsteins Bagels              26
Crossroads                    18
Runk Dining Hall              17
The Castle                    16
N2Go                          11
Dryer                         10
Washer                        10
Chick-Fil-A                   10
O-Hill Deposit                 3
Import Loaction                3
Dumpling Food Truck            2
West Range Cafe                1
Farmer's Market - Tapingo      1
Five Guys                      1
Residential-DomeorSpecial      1
Subway                         1
Name: Adjusted_Loc, dtype: int64